# Import Modules


In [0]:
import tensorflow as tf
import numpy as np
import pickle
import os
import csv



# Function to get train data from pkl files

In [0]:
def get_train_data():
    
    # Open train images file and load pickle
    with open("train_image.pkl","rb") as train_image:
        X = np.array(pickle.load(train_image))

    # Open train labels file and load pickle
    with open("train_label.pkl","rb") as train_label:
        Y = np.array(pickle.load(train_label))
    
    Y = tf.keras.utils.to_categorical(Y)
    
    return X,Y 

# Function to randomize data

In [0]:
def randomize(X,Y):

    # Make list of indices of X and Y
    indices = list(range(len(X)))

    np.random.shuffle(indices)

    X = X[indices]
    Y = Y[indices]

    return X,Y

# Function to train and save the model


In [0]:
def train_and_save(X,Y,batch_size=30,epochs=1000,learning_rate=0.001):
  
  # Create, compile and return model
  def get_model():
    
    # Input layer
    inp = tf.keras.layers.Input((784,))
    
    # First block
    d1 = tf.keras.layers.Dense(196,activation=tf.nn.relu)(inp)
    d2 = tf.keras.layers.Dense(98,activation=tf.nn.relu)(d1)
    
    # Second Left block
    dL3 = tf.keras.layers.Dense(64,activation=tf.nn.relu)(d2)
    dL4 = tf.keras.layers.Dense(32,activation=tf.nn.relu)(dL3)
    
    # Second Right block
    dR3 = tf.keras.layers.Dense(128,activation=tf.nn.relu)(d2)
    dR4 = tf.keras.layers.Dense(32,activation=tf.nn.relu)(dR3)
    
    # Third block
    d5 = tf.keras.layers.concatenate([dL4,dR4])
    d6 = tf.keras.layers.Dense(32,activation=tf.nn.relu)(d5)
    out = tf.keras.layers.Dense(7,activation=tf.nn.relu)(d6)
    
    # Create and compile model
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    adadelta = tf.keras.optimizers.Adadelta(lr=2)
    model.compile(adadelta,loss="categorical_crossentropy",metrics=["accuracy"])
    tf.keras.utils.plot_model(model,to_file="model.png",show_shapes=True,rankdir="TB")
    model.summary()
    
    return model
  
  # Train model
  def train(model):
    checkpoint = tf.keras.callbacks.ModelCheckpoint("\weights\weights.{epoch:02d}-{loss:.2f}.hdf5", monitor='loss', verbose=1, save_best_only=True, mode='min')
    train_history = model.fit(x = X,y = Y,batch_size=batch_size,epochs=epochs,use_multiprocessing=True,callbacks=[checkpoint])
    model.save("trained_model.h5")
    return train_history
  
  model = get_model()
  return train(model)
    
    

# Main

In [73]:
if __name__ == "__main__":
  X,Y = get_train_data()
  X,Y = randomize(X,Y)
  train_history = train_and_save(X,Y)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 784)          0                                            
__________________________________________________________________________________________________
dense_90 (Dense)                (None, 196)          153860      input_15[0][0]                   
__________________________________________________________________________________________________
dense_91 (Dense)                (None, 98)           19306       dense_90[0][0]                   
__________________________________________________________________________________________________
dense_92 (Dense)                (None, 64)           6336        dense_91[0][0]                   
__________________________________________________________________________________________________
dense_94 (

# Function to predict testing data and save labels in csv file

In [0]:
def predict_test_data():
  
  # Convert one hot to labels
  def one_hot_to_labels(one_hot):
    labels = []
    for i in one_hot:
      ind = np.argmax(i)
      labels.append(ind)
    return np.array(labels)
  
  # Load trained model
  model = tf.keras.models.load_model("\weights\weights.501-0.17.hdf5")
  
  # Load test data
  with open("test_image.pkl","rb") as test_image:
    test_X = pickle.load(test_image)
  test_X = np.array(test_X)
  
  
  # Predict test data label
  test_Y = model.predict(test_X,use_multiprocessing=True)
  test_Y = one_hot_to_labels(test_Y)
  
  # Save predicted labels to csv file
  with open("shantanu_shinde.csv","w") as csv_file:
    writer = csv.writer(csv_file,delimiter=",")
    writer.writerow(["Test_image_index","predicted class"])
    for i,label in enumerate(test_Y):
      writer.writerow([i,label])

In [0]:
# Predict the test data and save labels in csv file
predict_test_data()